In [ ]:
# Expectation-Maximisation algorithm with Maximum Likelihood part

In [ ]:
class EM_ML:
    # number of actions(num_actions), number of states(num_states), discount parameter(gamma), number of runs in each episode (num_in_episode)
    def __init__(self, num_actions, num_states, gamma, num_in_episode):

        self.num_actions = num_actions
        self.num_states = num_states
        self.gamma = gamma
        self.num_in_episode = num_in_episode
        self.reward = np.zeros((num_states, num_actions)) # r(s,a), which is represented by the (num_states, num_actions) matrix
        self.policy = np.random.random((num_actions, num_states)) # p(a|s), which is represented by (num_actions, num_states) matrix
        self.policy /= np.sum(self.policy, axis=0)  # normalisation, i.e. sum to 1 for each column
        self.message_left = np.zeros((num_in_episode, num_states)) # message passing from 1 to time t, where t is number of runs in each episode
        self.message_right = np.zeros((num_in_episode, num_states, num_actions)) # message passing from t to time 1
        self.alpha = np.ones((num_states, num_states, num_actions)) # prior for transition parameter theta, where all alphas are set to 1
        self.theta = self.alpha / np.sum(self.alpha, axis=0)  # normalisation, i.e. sum to 1 for given state and action
        self.start_dist = np.ones(num_states) # start distribution
        self.start_dist /= np.sum(self.start_dist) # normalisation, i.e. sum to 1 for all states
        self.last_state = None
        self.last_action = None


    # Update the transition parameter
    def update_theta(self):

        self.theta = self.alpha / np.sum(self.alpha, axis=0)  # normalisation, i.e. sum to 1 for given state and action


    # E-step 
    def Estep(self):

        for i in range(self.num_in_episode):
          # calculate left(alpha) message and right(beta) message
            if i == 0:
                self.message_right[i, :, :] = self.reward
                self.message_left[i, :] = self.start_dist
            else:
                self.message_right[i, :, :] = np.einsum('ij,ixy,ji->xy', self.message_right[i-1, :, :], self.theta, self.policy)
                self.message_left[i, :] = np.einsum('xij,ji,i->x', self.theta, self.policy, self.message_left[i-1, :])

    # M-step
    def Mstep(self):
        previous_policy = np.copy(self.policy)
        discounted_message_prod = np.zeros((self.num_actions, self.num_states))
        for tt in range(1, self.num_in_episode+1):
            discounted_message_prod += (self.gamma**(tt-1) * np.einsum('txa,tx->ax', np.flip(self.message_right[:tt, :, :], axis=0), self.message_left[:tt, :]))
        self.policy = self.policy* discounted_message_prod # update policy
        self.policy = self.policy + 1e-100  # give a small value for every useless actions under some states
        self.policy /= np.sum(self.policy, axis=0)  # normalisation
        differnece_policy = np.linalg.norm(previous_policy - self.policy, ord=1) # difference between old policy and updated policy
        return differnece_policy 


    # keep same starting situation for each episode
    def back(self):
        self.last_state = None
        self.last_action = None

    # iterate until satisfies convergence criteria
    def iteration(self, num_Iter=None, tol=0.01):
      self.update_theta()
      if num_Iter is None:  
        norm_diff = np.inf
        visualisation = tqdm()
        while norm_diff > tol: #loop until satisfies tolerance, i.e. convergence criteria
          self.Estep()
          norm_diff = self.Mstep()
          visualisation.update(1)
        visualisation.close()
      else:  # update for given number of steps
        for _ in tqdm(range(int(num_Iter))):
          self.Estep()
          self.Mstep()


    # choose action under policy distribution
    def make_action(self, state):
        action = np.random.choice(self.num_actions, p=self.policy[:, state]) # select action under current policy distribution
        if (self.last_state is not None) and (self.last_action is not None):
            self.alpha[state, self.last_state, self.last_action] += 1 # update count with observation 

        self.last_state = state
        self.last_action = action

        return action


    def tool(self):
      return self.policy

In [ ]:
# Expectation-Maximisation algorithm with Variational Bayes part

In [ ]:
class EM_VB:

    # number of actions(num_actions), number of states(num_states), discount parameter(gamma), number of runs in each episode (num_in_episode)
    def __init__(self, num_actions, num_states, gamma, num_in_episode):

        self.num_actions = num_actions
        self.num_states = num_states
        self.gamma = gamma
        self.num_in_episode = num_in_episode
        self.reward = np.zeros((num_states, num_actions)) # r(s,a), which is represented by the (num_states, num_actions) matrix
        self.policy = np.random.random((num_actions, num_states)) # p(a|s), which is represented by (num_actions, num_states) matrix
        self.policy /= np.sum(self.policy, axis=0)  # normalisation, i.e. sum to 1 for each column
        self.message_left = np.zeros((num_in_episode, num_states)) # message passing from 1 to time t, where t is number of runs in each episode (alpha_{\tau})
        self.message_right = np.zeros((num_in_episode, num_states, num_actions)) # message passing from t to time 1 (beta_{t-\tau})
        self.alpha = np.ones((num_states, num_states, num_actions)) # prior for transition parameter theta, where all alphas are set to 1
        self.theta = self.alpha / np.sum(self.alpha, axis=0)  # normalisation, i.e. sum to 1 for given state and action
        self.start_dist = np.ones(num_states) # start distribution
        self.start_dist /= np.sum(self.start_dist) # normalisation, i.e. sum to 1 for all states
        self.last_state = None
        self.last_action = None

    # E-step under Variational Bayes
    def Estep(self):

        local_message_left = np.zeros((self.num_in_episode-1, self.num_states)) # local message from 1 to time t-1, where t is number of runs in each episode (a_{\tau})
        local_message_right = np.zeros((self.num_in_episode-1, self.num_states))  # local message from t-1 to time 1, where t is number of runs in each episode (b_{t-\tau-1})
        tilde_r = np.zeros((self.num_states, self.num_states, self.num_actions)) #\tilde{r}, which is tensor of (num_states, num_states, num_actions)
        for _ in range(10): 
            q_distribution = np.zeros((self.num_states, self.num_states, self.num_actions, self.num_in_episode-1, self.num_in_episode-1)) # initialise q distribution, q(s_{\tau+1}, s_{\tau}, a_{\tau}, t)
            log_q_theta =  digamma(self.alpha+tilde_r) - digamma(np.sum(self.alpha+tilde_r, axis=0)) # using standard digamma to calculate expectation of log_\theta under distribution q_\theta
            exp_log_q_theta =  np.exp(log_q_theta)
            
            for i in range(self.num_in_episode-1): 
                if i == 0:
                    local_message_right[i, :] = np.einsum('as,sa->s', self.policy,self.reward)
                    local_message_left[i, :] = self.start_dist
                else:
                    local_message_right[i, :] = np.einsum('i,ijk,kj->j',local_message_right[i-1, :], exp_log_q_theta, self.policy)
                    local_message_left[i, :] = np.einsum('xij,ji,i->x', exp_log_q_theta, self.policy, local_message_left[i-1, :])

            for tt in range(1, self.num_in_episode):
              # update q distribution by using local message 
                q_distribution[:, :, :, 0:tt, tt-1] = self.gamma**(tt) * np.einsum('tp,ap,npa,tn->npat', local_message_left[:tt, :], self.policy, exp_log_q_theta, np.flip(local_message_right[:tt, :], axis=0))
            q_distribution /= np.sum(q_distribution) # normalisation, i.e. sum to 1

            tilde_r = np.einsum('npaut->npa', q_distribution) # calculation of \tilde_r

        for i in range(self.num_in_episode):
          # update global message
            if i == 0:
                self.message_right[i, :, :] = self.reward
                self.message_left[i, :] = self.start_dist
            else:
                self.message_right[i, :, :] = np.einsum('ij,ixy,ji->xy', self.message_right[i-1, :, :], exp_log_q_theta, self.policy)
                self.message_left[i, :] = np.einsum('xij,ji,i->x', exp_log_q_theta, self.policy, self.message_left[i-1, :])


    # M-step, its formula is same as normal EM
    def Mstep(self):
        previous_policy = np.copy(self.policy)
        discounted_message_prod = np.zeros((self.num_actions, self.num_states))
        for tt in range(1, self.num_in_episode+1):
            discounted_message_prod += (self.gamma**(tt-1) * np.einsum('txa,tx->ax', np.flip(self.message_right[:tt, :, :], axis=0), self.message_left[:tt, :]))
        self.policy = self.policy* discounted_message_prod # update policy
        self.policy /= np.sum(self.policy, axis=0)  # normalisation
        differnece_policy = np.linalg.norm(previous_policy - self.policy, ord=1) # difference between old policy and updated policy
        return differnece_policy 

    # keep same starting situation for each episode
    def back(self):
        self.last_state = None
        self.last_action = None


    # iterate until satisfies convergence criteria
    def iteration(self, num_Iter=None, tol=0.01):
      if num_Iter is None:  
        norm_diff = np.inf
        visualisation = tqdm()
        while norm_diff > tol: #loop until satisfies tolerance, i.e. convergence criteria
          self.Estep()
          norm_diff = self.Mstep()
          visualisation.update(1)
        visualisation.close()
      else:  # update for given number of steps
        for _ in tqdm(range(int(num_Iter))):
          self.Estep()
          self.Mstep()


    # choose action under policy distribution
    def make_action(self, state):
        action = np.random.choice(self.num_actions, p=self.policy[:, state]) # select action under current policy distribution
        if (self.last_state is not None) and (self.last_action is not None):
            self.alpha[state, self.last_state, self.last_action] += 1 # update count with observation 

        self.last_state = state
        self.last_action = action

        return action

    def tool(self):
      return self.policy

In [ ]:
# Dyna-Q algorithm

In [ ]:
# table model for the environment
class TabularModel(object):

  def __init__(self, number_of_states, number_of_actions):
    self.table_for_next_state = np.zeros((number_of_states,number_of_actions))
    self.table_for_reward = np.zeros((number_of_states,number_of_actions))
    self.table_for_discount = np.zeros((number_of_states,number_of_actions))

  def next_state(self, s, a):
    return self.table_for_next_state[s,a]
  
  def reward(self, s, a):
    return self.table_for_reward[s,a]

  def discount(self, s, a):
    return self.table_for_discount[s,a]
  
  def transition(self, state, action):
    return (
        self.reward(state, action), 
        self.discount(state, action),
        self.next_state(state, action))
  
  def update(self, state, action, reward, discount, next_state):
    self.table_for_next_state[state,action] = next_state
    self.table_for_reward[state,action] = reward
    self.table_for_discount[state,action] = discount

In [ ]:
# epsilon-greedy
def epsilon_greedy(q_values, state):
  if 0.3 < np.random.random():
    maxq = np.max(q_values[state,:])
    matches = [i for i in range(q_values.shape[1]) if q_values[state, i] == maxq]
    return np.random.choice(matches)
  else:
    return np.random.randint(np.array(q_values).shape[-1])

In [ ]:
class DynaQ(object):

  def __init__(
      self,  number_of_actions, number_of_states, initial_state, 
      behaviour_policy, Q, num_offline_updates=0, step_size=0.1):
    
    self._state = initial_state
    self._behaviour_policy = behaviour_policy
    self._num_offline_updates = num_offline_updates
    self._lr = step_size
    self._Q = Q
    self._action = self._behaviour_policy(self._Q, self._state)
    self._model = TabularModel(number_of_states, number_of_actions)
    self._replay_buffer = []  
    
  # action-value function
  def q_values(self):
    return self._Q

  def step(self, reward, discount, next_state):
    s = self._state
    a = self._action
    r = reward
    g = discount
    next_s = next_state

    self._Q[s,a] += self._lr * (r + g*np.max(self._Q[next_s,:]) - self._Q[s,a])
    self._replay_buffer.append((s,a))
    self._model.update(s, a, r, g, next_s)
    # offline updates
    for n in range(self._num_offline_updates):
      index = np.random.randint(len(self._replay_buffer))
      s, a = self._replay_buffer[index]
      r, g, next_s = self._model.transition(s, a)
      self._Q[s,a] += self._lr * (r + g*np.max(self._Q[int(next_s),:]) - self._Q[s,a])

    next_action = self._behaviour_policy(self._Q, self._state)
    self._state = next_state
    self._action = next_action

    return self._action

In [ ]:
!pip install pycolab

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from matplotlib import pyplot as plt
import seaborn as sns
import basic
import problem
import random
import grid_world
from scipy.special import digamma

In [ ]:
# chain problem

In [ ]:
# Environment settings for chain problem
gamma = 0.8
num_actions = 2
num_states = 5
reward_table = np.array([[0, 2],[0, 2],[0, 2],[0, 2], [10, 2]])
num_in_episode = 100
num_episodes = 20
num_experiments = 50
start_dist = np.array([1, 0, 0, 0, 0])

In [ ]:
reward_hist_each_experiment_EM_ML = []
learned_policy_EM_ML = []
for _ in tqdm(range(num_experiments)):
    emml = EM_ML(num_actions, num_states, gamma, num_in_episode)
    emml.reward = np.copy(reward_table)
    emml.update_theta()
    emml.start_dist = np.copy(start_dist)
    reward_hist_each_episode_EM_ML = []
    for _ in range(num_episodes):
        
        culmulative_reward = 0
        emml.back()
        case = problem.chain()
        observation = case.its_showtime()
        for _ in range(num_in_episode):
            action = emml.make_action(basic.location(observation, char='O'))
            observation = case.play(action)
            if not observation[1] is None:
                culmulative_reward += observation[1]
        #learn after the end of episode
        emml.iteration(tol=0.01)
        # add average reward to list
        reward_hist_each_episode_EM_ML.append(culmulative_reward/num_in_episode)
        # quit game
        case.play(5)
    learned_policy_EM_ML.append(emml.tool())
    reward_hist_each_experiment_EM_ML.append(reward_hist_each_episode_EM_ML)

In [ ]:
reward_hist_each_experiment_EM_VB = []
learned_policy_EM_VB = []
for _ in tqdm(range(num_experiments)):
    emvb = EM_VB(num_actions, num_states, gamma, num_in_episode)
    emvb.reward = np.copy(reward_table)
    emvb.start_dist = np.copy(start_dist)
    reward_hist_each_episode_EM_VB = []
    for _ in range(num_episodes):
        
        culmulative_reward = 0
        emvb.back()
        case = problem.chain()
        observation = case.its_showtime()
        for _ in range(num_in_episode):
            action = emvb.make_action(basic.location(observation, char='O'))
            observation = case.play(action)
            if not observation[1] is None:
                culmulative_reward += observation[1]
        #learn after the end of episode
        emvb.iteration(tol=0.01)
        # add average reward to list
        reward_hist_each_episode_EM_VB.append(culmulative_reward/num_in_episode)
        # quit game
        case.play(5)
    learned_policy_EM_VB.append(emvb.tool())
    reward_hist_each_experiment_EM_VB.append(reward_hist_each_episode_EM_VB)

In [ ]:
# Notice the combination of num_offline, epsilon-greedy, step_size, discount
reward_hist_each_experiment_DynaQ = []
learned_Q = []
for _ in tqdm(range(50)):
  Q = np.random.random((5,2))
  agent = DynaQ(2, 5, 0, epsilon_greedy, Q, num_offline_updates=30, step_size=0.1)
  mean_reward = 0.
  reward_hist_each_episode_DynaQ = []
  #reward_hist_each_episode_DynaQ.append(mean_reward)
  case = problem.chain()
  observation = case.its_showtime()
  observation = case.play(0)
  for i in range(100):
    r = observation[1]
    if observation[1] == None:
      r = 0
    ns = basic.location(observation, char='O')
    action = agent.step(reward = r, discount = 0.8, next_state = ns)
    observation = case.play(action)
    mean_reward += (r - mean_reward)/(i + 1.)
    print(r)
  reward_hist_each_episode_DynaQ.append(mean_reward)
  for __ in range(19):
    agent = DynaQ(2, 5, 0, epsilon_greedy, Q, num_offline_updates=30, step_size=0.1)
    case = problem.chain()
    observation = case.its_showtime()
    observation = case.play(0)
    mean_reward = 0.
    for i in range(100):
      r = observation[1]
      if observation[1] == None:
        r = 0
      ns = basic.location(observation, char='O')
      action = agent.step(reward = r, discount = 0.8, next_state = ns)
      observation = case.play(action)
      mean_reward += (r - mean_reward)/(i + 1.)
    #print(r)
    reward_hist_each_episode_DynaQ.append(mean_reward)
  learned_Q.append(Q)
  reward_hist_each_experiment_DynaQ.append(reward_hist_each_episode_DynaQ)

In [ ]:
#plot for experiment results

In [ ]:
import matplotlib.patches as mpatches
red_patch = mpatches.Patch(color='red', label='EM-ML')
black_patch = mpatches.Patch(color='black', label='EM-VB')
blue_patch = mpatches.Patch(color='blue', label='Dyna-Q')

In [ ]:
emml_reward_data = np.array(reward_hist_each_experiment_EM_ML)
emvb_reward_data = np.array(reward_hist_each_experiment_EM_VB)
DynaQ_reward_data = np.array(reward_hist_each_experiment_DynaQ)

In [ ]:
log = pd.DataFrame(columns=['episodes','average reward','Algorithm'])
for nx in range(50):
    for ne in range(20):
        log=log.append(pd.DataFrame({'episodes': ne+1,
                                    'average reward': emml_reward_data[nx, ne],
                                    'Algorithm': "EM-ML"}, index=[log.size+1]))
log1 = pd.DataFrame(columns=['episodes','average reward','Algorithm'])
for nx in range(50):
    for ne in range(20):
        log1=log1.append(pd.DataFrame({'episodes': ne+1,
                                    'average reward': emvb_reward_data[nx, ne],
                                    'Algorithm': "EM-VB"}, index=[log1.size+1]))
log2 = pd.DataFrame(columns=['episodes','average reward','Algorithm'])
for nx in range(50):
    for ne in range(20):
        log2=log2.append(pd.DataFrame({'episodes': ne+1,
                                    'average reward': DynaQ_reward_data[nx, ne],
                                    'Algorithm': "Dyna-Q"}, index=[log2.size+1]))

In [ ]:
sns.set_style('white')
A = sns.pointplot(x="episodes", y="average reward", data=log, color="red", label = "EM-ML", ci = 90)
B = sns.pointplot(x="episodes", y="average reward", data=log1, color="black", label = "EM-VB", ci = 90)
C = sns.pointplot(x="episodes", y="average reward", data=log2, color= "blue", label = "Dyna-Q", ci = 90)
plt.legend(handles=[red_patch, black_patch, blue_patch])
plt.xlabel('episodes')
plt.ylabel('average reward')
plt.ylim(1,3.4)
plt.title('EM-ML, EM-VB and Dyna-Q on chain problem with confidence interval')
plt.savefig('ENV1_with_ci')
plt.show()

In [ ]:
A = sns.pointplot(x="episodes", y="average reward", data=log, color="red", label = "EM-ML", ci = None)
B = sns.pointplot(x="episodes", y="average reward", data=log1, color="black", label = "EM-VB", ci= None)
C = sns.pointplot(x="episodes", y="average reward", data=log2, color= "blue", label = "Dyna-Q", ci = None)
plt.legend(handles=[red_patch, black_patch, blue_patch])
plt.xlabel('episodes')
plt.ylabel('average reward')
plt.ylim(1,3.4)
plt.title('EM-ML, EM-VB and Dyna-Q on chain problem')
plt.savefig('ENV1_without_ci')
plt.show()

In [ ]:
# increase the number of episodes, then run the reward_hist_each_experiment_DynaQ result with this code
from matplotlib.pyplot import figure
figure(figsize=(9, 6), dpi=100)
DynaQ_reward_data = np.array(reward_hist_each_experiment_DynaQ)
C = sns.pointplot(x="episodes", y="average reward", data=log3, color= "blue",label = "Dyna-Q",ci = 90)
plt.title('Dyna-Q with larger number of episodes on chain problem')
plt.savefig('Dyna_longer')
plt.show()

In [ ]:
# Grid world problem

In [ ]:
import grid_world

In [ ]:
# experiment setting for the grid world problem
grid = grid_world.make_game()
result = grid.its_showtime()
num_experiments = 30
grid_param = grid_world.get_params()
num_states = grid_param['num_states']
num_actions = grid_param['num_actions']
num_in_episode = 100
gamma = 0.9
num_episodes = 20

# Notice that the reward as 0 for moving to non-goal state and the reward as 50 for moving to goal state
reward = np.zeros((num_states, num_actions))
goal_loc = basic.location(result, 'X')
reward[goal_loc+1, 0] = 50
reward[goal_loc-1, 1] = 50
reward[goal_loc+11, 2] = 50
reward[goal_loc-11, 3] = 50
start_loc = basic.location(result, 'O')
start_dist = np.zeros(num_states)
start_dist[start_loc] = 1

# transition count
alpha = np.zeros((num_states, num_states, num_actions))+0.1
for state in range(num_states):
    # move to next state if it is not a wall
    if not basic.is_wall(state, result):
        if not basic.is_wall(state-1, result):
            alpha[state-1, state, 0] = 1
        else:
            alpha[state, state, 0] = 1
        if not basic.is_wall(state+1, result):
            alpha[state+1, state, 1] = 1
        else:  
            alpha[state, state, 1] = 1
        if not basic.is_wall(state-11, result):
            alpha[state-11, state, 2] = 1
        else:  
            alpha[state, state, 2] = 1
        if not basic.is_wall(state+11, result):
            alpha[state+11, state, 3] = 1
        else:
            alpha[state, state, 3] = 1
    # if current state is wall,  stay inside wall no matter which action
    else: 
        alpha[state, state, :] = 1

In [ ]:
# initialize policy to sub-optimal
emml_policy = np.ones((num_actions, num_states))
emml_policy[3, 24] = 10
emml_policy[3, 35] = 10
emml_policy[3, 46] = 10
emml_policy[1, 57] = 10
emml_policy[1, 58] = 10
emml_policy[1, 59] = 10
emml_policy[1, 60] = 10
emml_policy[1, 61] = 10
emml_policy[1, 62] = 10
emml_policy[2, 63] = 10
emml_policy[2, 52] = 10
emml_policy[2, 41] = 10
emml_policy[2, 30] = 10
emml_policy[2, 19] = 10
emml_policy /= np.sum(emml_policy, axis=0)  # normalise to make distribution

reward_hist_each_experiment_EM_ML = []
learned_policy_EM_ML = []
for _ in tqdm(range(num_experiments)):
    emml = EM_ML(num_actions, num_states,gamma,num_in_episode)
    emml.alpha = np.copy(alpha)
    emml.update_theta()
    emml.reward = np.copy(reward)
    emml.start_dist = np.copy(start_dist)
    emml.policy = np.copy(emml_policy)
    reward_hist_each_episodes_EM_ML = []
    for _ in range(num_episodes):
        cumulative_reward = 0
        emml.back()
        game = grid_world.make_game()
        obs = game.its_showtime()
        for _ in range(num_in_episode):
            action = emml.make_action(basic.location(obs, char='O'))
            obs = game.play(action)
            if not obs[1] is None:
                cumulative_reward += obs[1]  
        emml.iteration(tol=0.01)
        reward_hist_each_episodes_EM_ML.append(cumulative_reward/num_in_episode)
        game.play(5)
        print(cumulative_reward/num_in_episode)
    learned_policy_EM_ML.append(emml.tool())
    reward_hist_each_experiment_EM_ML.append(reward_hist_each_episodes_EM_ML)

In [ ]:
# Learned policy 
np.mean(np.array(learned_policy_EM_ML),axis =0)[:,24]
np.mean(np.array(learned_policy_EM_ML),axis =0)[:,23]
np.mean(np.array(learned_policy_EM_ML),axis =0)[:,12]
np.mean(np.array(learned_policy_EM_ML),axis =0)[:,25]
np.mean(np.array(learned_policy_EM_ML),axis =0)[:,35]
np.mean(np.array(learned_policy_EM_ML),axis =0)[:,36]
np.mean(np.array(learned_policy_EM_ML),axis =0)[:,13]
np.mean(np.array(learned_policy_EM_ML),axis =0)[:,14]
np.mean(np.array(learned_policy_EM_ML),axis =0)[:,15]
np.mean(np.array(learned_policy_EM_ML),axis =0)[:,16]
np.mean(np.array(learned_policy_EM_ML),axis =0)[:,17]
np.mean(np.array(learned_policy_EM_ML),axis =0)[:,28]
np.mean(np.array(learned_policy_EM_ML),axis =0)[:,29]
np.mean(np.array(learned_policy_EM_ML),axis =0)[:,18]
np.mean(np.array(learned_policy_EM_ML),axis =0)[:,19]
np.mean(np.array(learned_policy_EM_ML),axis =0)[:,30]
np.mean(np.array(learned_policy_EM_ML),axis =0)[:,31]
np.mean(np.array(learned_policy_EM_ML),axis =0)[:,41]

In [ ]:
# initialize policy to local minima
emvb_policy = np.ones((num_actions, num_states))
emvb_policy[3, 24] = 10
emvb_policy[3, 35] = 10
emvb_policy[3, 46] = 10
emvb_policy[1, 57] = 10
emvb_policy[1, 58] = 10
emvb_policy[1, 59] = 10
emvb_policy[1, 60] = 10
emvb_policy[1, 61] = 10
emvb_policy[1, 62] = 10
emvb_policy[2, 63] = 10
emvb_policy[2, 52] = 10
emvb_policy[2, 41] = 10
emvb_policy[2, 30] = 10
emvb_policy[2, 19] = 10
emvb_policy /= np.sum(emvb_policy, axis=0)  # normalize to make dist.
reward_hist_each_experiment_EM_VB = []
learned_policy_EM_VB = []
for _ in tqdm(range(num_experiments)):
    emvb = EM_VB(num_actions, num_states,gamma,num_in_episode)
    emvb.alpha = np.copy(alpha)
    emvb.reward = np.copy(reward)
    emvb.start_dist = np.copy(start_dist)
    emvb.policy = np.copy(emvb_policy)
    reward_hist_each_episodes_EM_VB = []
    for _ in range(num_episodes):
        cumulative_reward = 0
        emvb.back()
        game = grid_world.make_game()
        obs = game.its_showtime()
        for _ in range(num_in_episode):
            action = emvb.make_action(basic.location(obs, char='O'))
            obs = game.play(action)
            if not obs[1] is None:
                cumulative_reward += obs[1]  
        emvb.iteration(tol=0.01)
        reward_hist_each_episodes_EM_VB.append(cumulative_reward/num_in_episode)
        game.play(5)
        print(cumulative_reward/num_in_episode)
    learned_policy_EM_VB.append(vbem.tool())
    reward_hist_each_experiment_EM_VB.append(reward_hist_each_episodes_EM_VB)

In [ ]:
# Learned policy 
np.mean(np.array(learned_policy_EM_VB),axis =0)[:,24]
np.mean(np.array(learned_policy_EM_VB),axis =0)[:,23]
np.mean(np.array(learned_policy_EM_VB),axis =0)[:,12]
np.mean(np.array(learned_policy_EM_VB),axis =0)[:,25]
np.mean(np.array(learned_policy_EM_VB),axis =0)[:,35]
np.mean(np.array(learned_policy_EM_VB),axis =0)[:,36]
np.mean(np.array(learned_policy_EM_VB),axis =0)[:,13]
np.mean(np.array(learned_policy_EM_VB),axis =0)[:,14]
np.mean(np.array(learned_policy_EM_VB),axis =0)[:,15]
np.mean(np.array(learned_policy_EM_VB),axis =0)[:,16]
np.mean(np.array(learned_policy_EM_VB),axis =0)[:,17]
np.mean(np.array(learned_policy_EM_VB),axis =0)[:,28]
np.mean(np.array(learned_policy_EM_VB),axis =0)[:,29]
np.mean(np.array(learned_policy_EM_VB),axis =0)[:,18]
np.mean(np.array(learned_policy_EM_VB),axis =0)[:,19]
np.mean(np.array(learned_policy_EM_VB),axis =0)[:,30]
np.mean(np.array(learned_policy_EM_VB),axis =0)[:,31]
np.mean(np.array(learned_policy_EM_VB),axis =0)[:,41]

In [ ]:
reward_hist_each_experiment_DynaQ = []
learned_Q = []
for _ in tqdm(range(30)):
  Q = np.zeros((99,4))+0.1
  Q[24,3] = 10
  Q[35,3] = 10
  Q[46,3] = 10
  Q[57,1] = 10
  Q[58,1] = 10
  Q[59,1] = 10
  Q[60,1] = 10
  Q[61,1] = 10
  Q[62,1] = 10
  Q[63,1] = 10
  Q[52,2] = 10
  Q[41,2] = 10
  Q[30,2] = 10
  Q[19,2] = 10
  agent = DynaQ(4, 99, 24, epsilon_greedy, Q, num_offline_updates=100, step_size=0.2)
  mean_reward = 0.
  reward_hist_each_episode_DynaQ = []
  #reward_hist_each_episode_DynaQ.append(mean_reward)
  case = grid_world.make_game()
  observation = case.its_showtime()
  observation = case.play(0)
  for i in range(100):
    r = observation[1]
    if observation[1] == None:
      r = 0
    ns = basic.location(observation, char='O')
    action = agent.step(reward = r, discount = 0.9, next_state = ns)
    observation = case.play(action)
    mean_reward += (r - mean_reward)/(i + 1.)

  reward_hist_each_episode_DynaQ.append(mean_reward)
  for __ in range(19):
    agent = DynaQ(4, 99, 24, epsilon_greedy, Q, num_offline_updates=100, step_size=0.2)
    case =  grid_world.make_game()
    observation = case.its_showtime()
    observation = case.play(0)
    mean_reward = 0.
    for i in range(100):
      r = observation[1]
      if observation[1] == None:
        r = 0
      ns = basic.location(observation, char='O')
      action = agent.step(reward = r, discount = 0.9, next_state = ns)
      observation = case.play(action)
      mean_reward += (r - mean_reward)/(i + 1.)
    reward_hist_each_episode_DynaQ.append(mean_reward)
  learned_Q.append(Q)
  reward_hist_each_experiment_DynaQ.append(reward_hist_each_episode_DynaQ)
  

In [ ]:
# learned action-value function
np.mean(np.array(learned_Q),axis =0)[24,:]
np.mean(np.array(learned_Q),axis =0)[13,:]
np.mean(np.array(learned_Q),axis =0)[14,:]
np.mean(np.array(learned_Q),axis =0)[15,:]
np.mean(np.array(learned_Q),axis =0)[16,:]
np.mean(np.array(learned_Q),axis =0)[17,:]
np.mean(np.array(learned_Q),axis =0)[18,:]
np.mean(np.array(learned_Q),axis =0)[19,:]
np.mean(np.array(learned_Q),axis =0)[30,:]
np.mean(np.array(learned_Q),axis =0)[31,:]
np.mean(np.array(learned_Q),axis =0)[41,:]
np.mean(np.array(learned_Q),axis =0)[29,:]
np.mean(np.array(learned_Q),axis =0)[28,:]
np.mean(np.array(learned_Q),axis =0)[23,:]
np.mean(np.array(learned_Q),axis =0)[25,:]
np.mean(np.array(learned_Q),axis =0)[35,:]

In [ ]:
import matplotlib.patches as mpatches
red_patch = mpatches.Patch(color='red', label='EM-ML')
black_patch = mpatches.Patch(color='black', label='EM-VB')
blue_patch = mpatches.Patch(color='blue', label='Dyna-Q')

In [ ]:
emml_reward_data = np.array(reward_hist_each_experiment_EM_ML)
emvb_reward_data = np.array(reward_hist_each_experiment_EM_VB)
DynaQ_reward_data = np.array(reward_hist_each_experiment_DynaQ)

In [ ]:
log = pd.DataFrame(columns=['episodes','average reward','Algorithm'])
for nx in range(50):
    for ne in range(20):
        log=log.append(pd.DataFrame({'episodes': ne+1,
                                    'average reward': emml_reward_data[nx, ne],
                                    'Algorithm': "EM-ML"}, index=[log.size+1]))
log1 = pd.DataFrame(columns=['episodes','average reward','Algorithm'])
for nx in range(50):
    for ne in range(20):
        log1=log1.append(pd.DataFrame({'episodes': ne+1,
                                    'average reward': emvb_reward_data[nx, ne],
                                    'Algorithm': "EM-VB"}, index=[log1.size+1]))
log2 = pd.DataFrame(columns=['episodes','average reward','Algorithm'])
for nx in range(50):
    for ne in range(20):
        log2=log2.append(pd.DataFrame({'episodes': ne+1,
                                    'average reward': DynaQ_reward_data[nx, ne],
                                    'Algorithm': "Dyna-Q"}, index=[log2.size+1]))

In [ ]:
sns.set_style('white')
A = sns.pointplot(x="episodes", y="average reward", data=log, color="red", label = "EM-ML", ci = 90)
B = sns.pointplot(x="episodes", y="average reward", data=log1, color="black", label = "EM-VB", ci = 90)
C = sns.pointplot(x="episodes", y="average reward", data=log2, color= "blue", label = "Dyna-Q", ci = 90)
plt.legend(handles=[red_patch, black_patch, blue_patch])
plt.xlabel('episodes')
plt.ylabel('average reward')
plt.title('EM-ML, EM-VB and Dyna-Q on chain problem with confidence interval')
plt.savefig('ENV2_with_ci')
plt.show()

In [ ]:
A = sns.pointplot(x="episodes", y="average reward", data=log, color="red", label = "EM-ML", ci = None)
B = sns.pointplot(x="episodes", y="average reward", data=log1, color="black", label = "EM-VB", ci= None)
C = sns.pointplot(x="episodes", y="average reward", data=log2, color= "blue", label = "Dyna-Q", ci = None)
plt.legend(handles=[red_patch, black_patch, blue_patch])
plt.xlabel('episodes')
plt.ylabel('average reward')
plt.title('EM-ML, EM-VB and Dyna-Q on chain problem')
plt.savefig('ENV2_without_ci')
plt.show()